Downloading models

In [1]:
!git clone https://github.com/open-mmlab/mmsr.git

Cloning into 'mmsr'...
remote: Enumerating objects: 136, done.
remote: Total 136 (delta 0), reused 0 (delta 0), pack-reused 136
Receiving objects: 100% (136/136), 97.74 KiB | 575.00 KiB/s, done.
Resolving deltas: 100% (44/44), done.


Fix DCN compilation errors due to PyTorch v.1.5

In [2]:
! sed 's/AT_CHECK/TORCH_CHECK/g' /content/mmsr/codes/models/archs/dcn/src/deform_conv_cuda.cpp > /content/mmsr/codes/models/archs/dcn/src/tmp.cpp

! sed -e 's/data_col.data/data_col.data_ptr/g' -e 's/data_im.data/data_im.data_ptr/g' -e 's/data_mask.data/data_mask.data_ptr/g' -e 's/data_offset.data/data_offset.data_ptr/g' \
      -e 's/grad_im.data/grad_im.data_ptr/g' -e 's/grad_mask.data/grad_mask.data_ptr/g' -e 's/grad_offset.data/grad_offset.data_ptr/g' -e 's/pad_h, pad_h/pad_h, pad_w/g' \
      /content/mmsr/codes/models/archs/dcn/src/deform_conv_cuda_kernel.cu > /content/mmsr/codes/models/archs/dcn/src/tmp.cu

! mv /content/mmsr/codes/models/archs/dcn/src/tmp.cpp /content/mmsr/codes/models/archs/dcn/src/deform_conv_cuda.cpp
! mv /content/mmsr/codes/models/archs/dcn/src/tmp.cu /content/mmsr/codes/models/archs/dcn/src/deform_conv_cuda_kernel.cu

Setup DCN

In [3]:
! cd mmsr/codes/models/archs/dcn/; python setup.py develop

running develop
running egg_info
creating deform_conv.egg-info
writing deform_conv.egg-info/PKG-INFO
writing dependency_links to deform_conv.egg-info/dependency_links.txt
writing top-level names to deform_conv.egg-info/top_level.txt
writing manifest file 'deform_conv.egg-info/SOURCES.txt'
/usr/local/lib/python3.6/dist-packages/torch/utils/cpp_extension.py:304: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
writing manifest file 'deform_conv.egg-info/SOURCES.txt'
running build_ext
building 'deform_conv_cuda' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/src
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/torch/include -I/usr/local/li

In [ ]:
! pip install -r /content/mmsr/codes/requirements.txt

Download and export dataset

In [6]:
fileId = '1NB_n7dzheENpBOGA7kUvLaM5sarPs5RP'

import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

os.makedirs('dataset')
os.chdir('dataset')

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fileName = fileId + '.zip'
downloaded = drive.CreateFile({'id': fileId})
downloaded.GetContentFile(fileName)
ds = ZipFile(fileName)
ds.extractall()
os.remove(fileName)
print('Extracted zip file ' + fileName)

Extracted zip file 1NB_n7dzheENpBOGA7kUvLaM5sarPs5RP.zip


Train ESRGAN model

In [ ]:
# Upload a pretrained RRDB model
# Upload updated train_ESRGAN.yml options file

! cd /content/mmsr/codes/; python train.py -opt options/train/train_ESRGAN.yml

export CUDA_VISIBLE_DEVICES=0
Disabled distributed training.
20-06-21 10:51:04.387 - INFO:   name: ESRGANx4-hr
  use_tb_logger: True
  model: srgan
  distortion: sr
  scale: 4
  gpu_ids: [0]
  datasets:[
    train:[
      name: MicroTissue
      mode: LQGT
      dataroot_GT: /content/dataset/dataset/sliced
      dataroot_LQ: /content/dataset/dataset/train_LR
      use_shuffle: True
      n_workers: 6
      batch_size: 32
      GT_size: 128
      use_flip: True
      use_rot: True
      color: RGB
      phase: train
      scale: 4
      data_type: img
    ]
    val:[
      name: MicroTissue_val
      mode: LQGT
      dataroot_GT: /content/dataset/dataset/sliced-val
      dataroot_LQ: /content/dataset/dataset/val_LR
      phase: val
      scale: 4
      data_type: img
    ]
  ]
  network_G:[
    which_model_G: RRDBNet
    in_nc: 3
    out_nc: 3
    nf: 64
    nb: 23
    scale: 4
  ]
  network_D:[
    which_model_D: discriminator_vgg_128
    in_nc: 3
    nf: 64
  ]
  path:[
    pretrain_m

In [ ]:
# Upload test options

! cd /content/mmsr/codes/; python test.py -opt options/test/test_ESRGAN.yml

Train SRGAN model

In [ ]:
# Upload pretrained MSRGAN model
# Upload updated train_SRGAN.yml options file

! cd /content/mmsr/codes/; python train.py -opt options/train/train_SRGAN.yml

In [ ]:
! cd /content/mmsr/codes/; python test.py -opt options/test/test_SRGAN.yml

Download dataset for testing without GT:

In [ ]:
fileId = '1S19P_Ev3tDLejSPLT5J38XLBdZ8wTUkq'

import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

os.makedirs('testing-without-gt')
os.chdir('testing-without-gt')

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fileName = fileId + '.zip'
downloaded = drive.CreateFile({'id': fileId})
downloaded.GetContentFile(fileName)
ds = ZipFile(fileName)
ds.extractall()
os.remove(fileName)
print('Extracted zip file ' + fileName)

Extracted zip file 1S19P_Ev3tDLejSPLT5J38XLBdZ8wTUkq.zip


In [ ]:
# Upload updated file - without ground truth path

! cd /content/mmsr/codes/; python test.py -opt options/test/test_SRGAN.yml

export CUDA_VISIBLE_DEVICES=0
20-06-20 14:17:27.383 - INFO:   name: SRGANx4-hr
  suffix: None
  model: sr
  distortion: sr
  scale: 4
  crop_border: None
  gpu_ids: [0]
  datasets:[
    test_1:[
      name: set3
      mode: LQ
      dataroot_LQ: /content/dataset/testing-without-gt/testing-dataset
      phase: test
      scale: 4
      data_type: img
    ]
  ]
  network_G:[
    which_model_G: MSRResNet
    in_nc: 3
    out_nc: 3
    nf: 64
    nb: 16
    upscale: 4
    scale: 4
  ]
  path:[
    pretrain_model_G: ../experiments/4xSRGAN-hr/models/2000_G.pth
    root: /content/mmsr
    results_root: /content/mmsr/results/SRGANx4-hr
    log: /content/mmsr/results/SRGANx4-hr
  ]
  is_train: False

20-06-20 14:17:27.386 - INFO: Dataset [LQDataset - set3] is created.
20-06-20 14:17:27.386 - INFO: Number of test images in [set3]: 36
20-06-20 14:17:29.625 - INFO: Network G structure: DataParallel - MSRResNet, with parameters: 1,517,571
20-06-20 14:17:29.625 - INFO: MSRResNet(
  (conv_first): Con